<a href="https://colab.research.google.com/github/microprediction/winningnotebooks/blob/main/Luce_Axiom_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install transformers
!pip install winning
!pip install pandas
!pip install scipy

# Luce's Axiom (Implicit in Softmax)
Since it is a softmax,  we obviously can surgically force the transformer to obey Luce.

In [27]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import logging
from transformers import logging as transformers_logging
from transformers import pipeline
transformers_logging.set_verbosity_error()


# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

def fill_in_missing_word(sentence, exclude_words=None, top_k=20):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors='pt')
    input_ids = inputs['input_ids']

    # Find the index of the masked token
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

    # Get the model's predictions (logits)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits

    if exclude_words:
        # Get the IDs of the words to exclude
        exclude_ids = tokenizer.convert_tokens_to_ids(exclude_words)
        # Set their logits to a very low value
        logits[0, mask_token_index, exclude_ids] = -float('inf')

    # Apply softmax to get probabilities
    probs = torch.softmax(logits[0, mask_token_index], dim=-1).squeeze()

    # Get the top_k predictions
    top_k_probs, top_k_indices = torch.topk(probs, top_k, dim=-1)

    top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices.tolist())
    top_k_probs = top_k_probs.tolist()

    # Store the top predictions in a dictionary
    predictions = dict(zip(top_k_tokens, top_k_probs))

    return predictions

# Use the tokenizer's mask token in the sentence
sentence = f"The space-ship was so {tokenizer.mask_token} it was not possible to see what was behind it."

# First iteration: get original probabilities
original_predictions = fill_in_missing_word(sentence, top_k=20)

# Extract the top predicted word and its probability
first_word = max(original_predictions, key=original_predictions.get)
first_prob = original_predictions[first_word]

print("Original Top Predictions:")
for word, prob in original_predictions.items():
    print(f"{word}: {prob:.6f}")

print(f"\nFirst answer: '{first_word}', Probability: {first_prob:.6f}\n")

# Second iteration: remove the most probable word
new_predictions = fill_in_missing_word(sentence, exclude_words=[first_word], top_k=20)

print("New Top Predictions After Excluding the First Word:")
for word, prob in new_predictions.items():
    print(f"{word}: {prob:.6f}")

# Compare probabilities according to Luce Choice Axiom
print("\nComparing Ratios According to Luce Choice Axiom:\n")

# Calculate the sum of probabilities excluding the first word
sum_original_probs_excl_first = sum(original_predictions[word] for word in original_predictions if word != first_word)
sum_new_probs = sum(new_predictions.values())

print(f"Sum of original probabilities (excluding '{first_word}'): {sum_original_probs_excl_first:.6f}")
print(f"Sum of new probabilities: {sum_new_probs:.6f}")

# Verify that sums are approximately equal (should be close to 1 - first_prob)
print(f"1 - Probability of '{first_word}': {1 - first_prob:.6f}")

# Check ratios for each word in new_predictions
print("\nWord\tOriginal Prob\tExpected New Prob\tActual New Prob\tRatio (Actual/Expected)")
for word in new_predictions:
    original_prob = original_predictions.get(word, 0)
    expected_new_prob = original_prob / (1 - first_prob)
    actual_new_prob = new_predictions[word]
    ratio = actual_new_prob / expected_new_prob if expected_new_prob != 0 else 0
    print(f"{word}\t{original_prob:.6f}\t{expected_new_prob:.6f}\t{actual_new_prob:.6f}\t{ratio:.6f}")


Original Top Predictions:
large: 0.344735
small: 0.227211
massive: 0.106483
huge: 0.058782
big: 0.030036
close: 0.020969
dark: 0.020564
enormous: 0.017063
low: 0.010211
tiny: 0.010106
heavy: 0.009116
immense: 0.007942
tall: 0.007581
vast: 0.007423
high: 0.006501
narrow: 0.004375
powerful: 0.003378
quiet: 0.003095
thin: 0.003060
silent: 0.002666

First answer: 'large', Probability: 0.344735

New Top Predictions After Excluding the First Word:
small: 0.346745
massive: 0.162502
huge: 0.089707
big: 0.045837
close: 0.032000
dark: 0.031383
enormous: 0.026040
low: 0.015584
tiny: 0.015423
heavy: 0.013912
immense: 0.012120
tall: 0.011569
vast: 0.011328
high: 0.009920
narrow: 0.006677
powerful: 0.005156
quiet: 0.004724
thin: 0.004669
silent: 0.004069
black: 0.003872

Comparing Ratios According to Luce Choice Axiom:

Sum of original probabilities (excluding 'large'): 0.556562
Sum of new probabilities: 0.853237
1 - Probability of 'large': 0.655265

Word	Original Prob	Expected New Prob	Actual New P

# "Real" Choice
But what about making actual choices in context? Here we try to ask very similar questions to remove horses from a race.

In [156]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import pandas as pd

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

def fill_in_missing_word(sentence, exclude_words=None, top_k=20):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors='pt')
    input_ids = inputs['input_ids']

    # Find the index of the masked token
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

    # Get the model's predictions (logits)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits

    if exclude_words:
        # Get the IDs of the words to exclude
        exclude_ids = tokenizer.convert_tokens_to_ids(exclude_words)
        # Set their logits to a very low value
        logits[0, mask_token_index, exclude_ids] = -float('inf')

    # Apply softmax to get probabilities
    probs = torch.softmax(logits[0, mask_token_index], dim=-1)

    # Get the top_k predictions
    top_k_probs, top_k_indices = torch.topk(probs, top_k, dim=-1)

    # If there's only one mask token, adjust dimensions
    if top_k_indices.dim() == 2 and top_k_indices.size(0) == 1:
        top_k_indices = top_k_indices.squeeze(0)
        top_k_probs = top_k_probs.squeeze(0)

    top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices.tolist())
    top_k_probs = top_k_probs.tolist()

    # Store the top predictions in a dictionary
    predictions = dict(zip(top_k_tokens, top_k_probs))

    return predictions


# Luce Axiom Check
def luce_check(sentence1, sentence2):
    # Get probabilities from both sentences
    probs1 = fill_in_missing_word(sentence1, top_k=100)
    probs2 = fill_in_missing_word(sentence2, top_k=10)

    # Filter out words in sentence2 not present in sentence1
    common_tokens = set(probs1.keys()).intersection(set(probs2.keys()))

    # Create filtered dictionaries with common tokens
    probs1_filtered = {token: probs1[token] for token in common_tokens}
    probs2_filtered = {token: probs2[token] for token in common_tokens}

    # Store the original scores (unnormalized) before renormalization
    original_scores = {token: probs1[token] for token in probs1_filtered}


    # Renormalize probs2 so they sum to 1
    total_prob2 = sum(probs2_filtered.values())
    probs2_normalized = {token: prob / total_prob2 for token, prob in probs2_filtered.items()}

    # Renormalize probs1 so they sum to 1
    total_prob1 = sum(probs1_filtered.values())
    probs1_normalized = {token: prob / total_prob1 for token, prob in probs1_filtered.items()}


    # Merge both into a DataFrame for comparison
    df = pd.DataFrame({
        'Choice': list(probs1_normalized.keys()),
        'Original score': [original_scores[token] for token in probs1_normalized.keys()],
        'Implied score': list(probs1_normalized.values()),
        'Actual score': [probs2_normalized[token] for token in probs1_normalized.keys()]
    })

    # Add a column for the empirical / Luce ratio
    df['Ratio'] = df['Actual score'] / df['Implied score']
    df.sort_values('Implied score',inplace=True, ascending=False)

    return df


# Function to display results
def display_predictions(sentence, exclude_words=None):
    predictions = fill_in_missing_word(sentence, exclude_words=exclude_words)
    sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    print(f"Sentence: {sentence}")
    print("Top predictions:")
    for word, score in sorted_predictions:
        print(f"  {word}: {score:.4f}")
    print("\n")
    return sorted_predictions






/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Eating

In [157]:
sentence1 = f"My favourite type of fruit or vegetable is called a {tokenizer.mask_token} and I eat one of them every day."
sentence2 = f"My favourite type of fruit is called a {tokenizer.mask_token} and I eat one of them every day."
df = luce_check(sentence1, sentence2)
print(df)


       Choice  Original score  Implied score  Actual score     Ratio
9      banana        0.114603       0.266236      0.178440  0.670235
4       mango        0.059991       0.139366      0.249007  1.786718
2         nut        0.052449       0.121846      0.068018  0.558233
1       fruit        0.034807       0.080861      0.088250  1.091377
0       peach        0.032701       0.075967      0.077764  1.023651
6       berry        0.032125       0.074630      0.053912  0.722390
3        plum        0.029800       0.069228      0.067843  0.980001
7       lemon        0.027329       0.063488      0.058413  0.920065
5  strawberry        0.024800       0.057614      0.056728  0.984631
8      cherry        0.021852       0.050765      0.101623  2.001842


# States

In [158]:
sentence1 = f"My favourite state in the U.S. is {tokenizer.mask_token} and I try to visit once a year."
sentence2 = f"My favourite Western state in the U.S. is {tokenizer.mask_token} and I try to visit once a year."
df = luce_check(sentence1, sentence2)
print(df)

       Choice  Original score  Implied score  Actual score     Ratio
6  california        0.108853       0.298702      0.201740  0.675390
9     arizona        0.067399       0.184951      0.100027  0.540833
1       texas        0.055019       0.150977      0.076750  0.508360
8    colorado        0.031841       0.087376      0.078146  0.894368
7      oregon        0.028873       0.079231      0.085660  1.081137
5    oklahoma        0.019698       0.054052      0.066355  1.227612
3      nevada        0.016581       0.045499      0.101803  2.237464
4     montana        0.015908       0.043653      0.142662  3.268112
2       idaho        0.010460       0.028704      0.049947  1.740053
0     wyoming        0.009787       0.026856      0.096909  3.608484


# Countries

In [159]:
sentence1 = f"My favourite drink is {tokenizer.mask_token} and it tastes great."
sentence2 = f"My favourite alcoholic drink is {tokenizer.mask_token} and it tastes great."
df = luce_check(sentence1, sentence2)
print(df)

      Choice  Original score  Implied score  Actual score     Ratio
6      water        0.087032       0.196648      0.148692  0.756133
8     coffee        0.079112       0.178752      0.075745  0.423745
7        gin        0.061759       0.139543      0.169855  1.217224
1       wine        0.048430       0.109426      0.080789  0.738293
5        rum        0.034824       0.078685      0.134153  1.704928
2       beer        0.034671       0.078338      0.072892  0.930489
4  champagne        0.032639       0.073749      0.045811  0.621178
9      vodka        0.028924       0.065353      0.144282  2.207720
0       cola        0.019985       0.045155      0.051135  1.132432
3    bourbon        0.015203       0.034351      0.076646  2.231272


# Pets

In [160]:
sentence1 = f"My favorite type of pet or farm animal is a {tokenizer.mask_token}, and it is very loyal."
sentence2 = f"My favorite type of pet is a {tokenizer.mask_token}, and it is very loyal."
df = luce_check(sentence1, sentence2)
print(df)

      Choice  Original score  Implied score  Actual score     Ratio
9        dog        0.111114       0.227469      0.170574  0.749878
2        cat        0.071628       0.146635      0.270100  1.841989
5       wolf        0.061467       0.125834      0.082584  0.656294
6       bear        0.057627       0.117972      0.115044  0.975183
8       lion        0.044970       0.092062      0.127784  1.388024
3      tiger        0.040343       0.082590      0.041469  0.502112
0     rabbit        0.039527       0.080919      0.051705  0.638977
7  chihuahua        0.033485       0.068549      0.033833  0.493551
4    leopard        0.017254       0.035322      0.068246  1.932106
1      mouse        0.011063       0.022648      0.038661  1.706991


# Furniture

In [162]:
sentence1 = f"I need to buy a new piece of furniture or appliance called a {tokenizer.mask_token} for my house."
sentence2 = f"I need to buy a new piece of furniture called a {tokenizer.mask_token} for my house."
df = luce_check(sentence1, sentence2)
print(df)

      Choice  Original score  Implied score  Actual score     Ratio
2      piano        0.012473       0.219646      0.098736  0.449523
1       room        0.007054       0.124207      0.127745  1.028484
5  furniture        0.006571       0.115705      0.121403  1.049243
9      chair        0.005992       0.105506      0.114693  1.087075
4    bedroom        0.005213       0.091797      0.085581  0.932283
3        bed        0.004793       0.084400      0.106573  1.262715
8       sofa        0.004417       0.077779      0.103240  1.327352
7      table        0.004067       0.071618      0.088269  1.232504
6       desk        0.003359       0.059155      0.071658  1.211344
0    dresser        0.002850       0.050187      0.082102  1.635945


# Classics

In [101]:
sentence1 = f"They are studying modern or classical {tokenizer.mask_token} at the university."
sentence2 = f"They are studying classical {tokenizer.mask_token} at the university."
df = luce_check(sentence1, sentence2)
print(df)

        Choice  Implied score  Actual score      Ratio
8        music       0.573795      0.500643   0.872513
0    languages       0.189659      0.100392   0.529329
3   literature       0.071643      0.075768   1.057582
7      history       0.063840      0.026200   0.410404
4        dance       0.053870      0.046529   0.863729
5      studies       0.015291      0.031336   2.049256
9  composition       0.013594      0.018114   1.332457
6        greek       0.007472      0.023592   3.157258
1    philology       0.006550      0.150025  22.905527
2       ballet       0.004285      0.027400   6.394469


# Big cats

In [103]:
sentence1 = f"My favorite animal at the zoo is the {tokenizer.mask_token}, and I always visit its enclosure."
sentence2 = f"My favorite big cat at the zoo is the {tokenizer.mask_token}, and I always visit its enclosure."
df = luce_check(sentence1, sentence2)
print(df)

    Choice  Implied score  Actual score     Ratio
6     lion       0.546050      0.204367  0.374264
3    tiger       0.151210      0.201513  1.332673
5     bear       0.072922      0.080488  1.103761
4  leopard       0.068827      0.075231  1.093049
2  panther       0.063777      0.088980  1.395177
1      cat       0.062657      0.294430  4.699074
0    mouse       0.034558      0.054991  1.591258


# Getting Around

In [105]:
sentence1 = f"My preferred mode of transportation is the {tokenizer.mask_token}, it's very convenient."
sentence2 = f"My preferred public transportation is the {tokenizer.mask_token}, it's very convenient."
df = luce_check(sentence1, sentence2)
print(df)

    Choice  Implied score  Actual score     Ratio
0      car       0.293049      0.186921  0.637848
4      bus       0.221204      0.195267  0.882744
6     road       0.101527      0.046570  0.458697
5   subway       0.086666      0.210164  2.424987
2  airport       0.081799      0.089627  1.095710
3  highway       0.078798      0.054571  0.692536
7    train       0.058760      0.050261  0.855363
1    buses       0.033907      0.073449  2.166226
8  freeway       0.031542      0.046425  1.471831
9    metro       0.012748      0.046745  3.666779


# Holidays

In [107]:
sentence1 = f"My favorite holiday is {tokenizer.mask_token}, and I always celebrate it with family."
sentence2 = f"My favorite winter holiday is {tokenizer.mask_token}, and I always celebrate it with family."
df = luce_check(sentence1, sentence2)
print(df[:3])


         Choice  Implied score  Actual score     Ratio
7     christmas       0.854978      0.545480  0.638004
1  thanksgiving       0.092472      0.132594  1.433873
8     halloween       0.023282      0.030875  1.326135


# And some more...

In [110]:
# Example 1
sentence1 = f"My favorite city to visit is {tokenizer.mask_token}, and I go there every summer."
sentence2 = f"My favorite French city to visit is {tokenizer.mask_token}, and I go there every summer."
df = luce_check(sentence1, sentence2)
print(df)

   Choice  Implied score  Actual score     Ratio
0   paris       0.815324      0.840575  1.030970
2    nice       0.106383      0.150050  1.410467
1  geneva       0.078292      0.009375  0.119748


      Choice  Implied score  Actual score     Ratio
6    chicken       0.685280      0.147544  0.215305
4       rice       0.095997      0.071000  0.739607
2      china       0.077318      0.136574  1.766379
0    chinese       0.062546      0.334283  5.344620
5  cantonese       0.028176      0.079458  2.820050
3   japanese       0.022814      0.130745  5.730838
7       thai       0.021291      0.062383  2.929993
1      curry       0.006577      0.038014  5.779574


In [113]:
sentence1 = f"The best car brand, in my opinion, is {tokenizer.mask_token}."
sentence2 = f"The best German car brand, in my opinion, is {tokenizer.mask_token}."
df = luce_check(sentence1, sentence2)
print(df)

       Choice  Implied score  Actual score     Ratio
0         bmw       0.333629      0.606744  1.818619
4    mercedes       0.207512      0.110781  0.533857
3        ford       0.187307      0.026922  0.143733
2     ferrari       0.099664      0.033399  0.335118
6     porsche       0.082868      0.117889  1.422624
7     renault       0.058201      0.042578  0.731572
5  volkswagen       0.015427      0.037029  2.400227
1        audi       0.015393      0.024657  1.601792


# Programming languages

In [115]:

sentence1 = f"My favorite programming language is {tokenizer.mask_token}, I use it daily."
sentence2 = f"My favorite object-oriented programming language is {tokenizer.mask_token}, I use it daily."
df = luce_check(sentence1, sentence2)
print(df)

   Choice  Implied score  Actual score     Ratio
6    java       0.334261      0.430050  1.286569
7       c       0.294080      0.267487  0.909570
1  python       0.140631      0.125992  0.895908
0     php       0.092112      0.024155  0.262237
5   basic       0.060059      0.017473  0.290924
4    html       0.040316      0.006228  0.154483
3  pascal       0.021915      0.100941  4.606001
2     sql       0.008406      0.013398  1.593861
8    ruby       0.008220      0.014276  1.736783


# Starting the day?

In [116]:
sentence1 = f"I like to drink {tokenizer.mask_token} in the morning to start my day."
sentence2 = f"I like to drink hot {tokenizer.mask_token} in the morning to start my day."
df = luce_check(sentence1, sentence2)
print(df)

   Choice  Implied score  Actual score     Ratio
3  coffee       0.549390      0.238140  0.433463
2   water       0.227673      0.657931  2.889812
1    beer       0.158892      0.008129  0.051160
4     tea       0.028318      0.059600  2.104672
5    milk       0.027269      0.006568  0.240859
0  drinks       0.008458      0.029632  3.503310


# Scientists

In [117]:
sentence1 = f"My favorite scientist is {tokenizer.mask_token}, they changed the world."
sentence2 = f"My favorite 19th Century scientist is {tokenizer.mask_token}, they changed the world."
df = luce_check(sentence1, sentence2)
print(df)

     Choice  Implied score  Actual score     Ratio
3  einstein       0.685723      0.186807  0.272423
0      here       0.106623      0.239498  2.246216
5       you       0.056903      0.091914  1.615287
9      back       0.030993      0.076372  2.464219
4      dead       0.028130      0.049693  1.766531
1        me       0.027055      0.085118  3.146138
8     right       0.022430      0.070561  3.145860
7   charles       0.017279      0.078950  4.569118
2      gone       0.012904      0.070766  5.483961
6        he       0.011961      0.050322  4.207103


# More pets

In [120]:
sentence1 = f"My preferred pet is a {tokenizer.mask_token}, they make great companions."
sentence2 = f"My preferred small pet is a {tokenizer.mask_token}, they make great companions."
df = luce_check(sentence1, sentence2)
print(df)


     Choice  Implied score  Actual score     Ratio
2       cat       0.155339      0.160443  1.032855
5      bear       0.141039      0.127901  0.906848
8      lion       0.130099      0.090659  0.696849
0      deer       0.127077      0.132946  1.046185
6      wolf       0.115865      0.095933  0.827969
3     tiger       0.103580      0.087945  0.849057
9       dog       0.089925      0.094879  1.055091
4  squirrel       0.060674      0.082661  1.362362
1     mouse       0.038420      0.068674  1.787468
7    parrot       0.037983      0.057961  1.525957


In [136]:
sentence1 = f"My favorite baby name is {tokenizer.mask_token}, and I always associate it with family."
sentence2 = f"My favorite girl's baby name is {tokenizer.mask_token}, and I always associate it with family."
df = luce_check(sentence1, sentence2)
print(df)

     Choice  Implied score  Actual score     Ratio
3      lily       0.203468      0.145201  0.713629
5     emily       0.150642      0.162862  1.081119
0       mia       0.112507      0.131783  1.171325
4     bella       0.111161      0.088390  0.795158
8     sarah       0.088915      0.094142  1.058783
7    rachel       0.076603      0.071505  0.933443
9      kate       0.076348      0.084366  1.105008
1    lauren       0.061055      0.072298  1.184147
6  brittany       0.060399      0.075525  1.250435
2     chloe       0.058902      0.073929  1.255132


In [135]:
sentence1 = f"My favorite baseball team are the {tokenizer.mask_token}, and I always associate them with success."
sentence2 = f"My favorite new york baseball team are the {tokenizer.mask_token}, and I always associate them with success."
df = luce_check(sentence1, sentence2)
print(df[:3])

    Choice  Implied score  Actual score     Ratio
5  yankees       0.264528      0.401384  1.517361
3   tigers       0.186640      0.017931  0.096070
2     mets       0.179004      0.433763  2.423203


In [149]:
sentence1 = f"My favorite age is {tokenizer.mask_token}, and my daughter is that age"
sentence2 = f"My favorite age for middle school kids is {tokenizer.mask_token}, and my daughter is that age"
df = luce_check(sentence1, sentence2)
print(df)

   Choice  Implied score  Actual score     Ratio
7   eight       0.155288      0.152468  0.981840
9  twelve       0.138472      0.139279  1.005830
4  eleven       0.128694      0.085072  0.661040
0   seven       0.113845      0.085769  0.753386
5    five       0.113015      0.084129  0.744404
3      16       0.094257      0.103968  1.103035
2     six       0.076396      0.089710  1.174276
1      12       0.066326      0.116064  1.749902
6       9       0.063752      0.068647  1.076778
8      13       0.049956      0.074894  1.499213
